In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib
import pickle

In [57]:
train = pd.read_csv('Train_maskedv2.csv')
test = pd.read_csv('Test_maskedv2.csv')
submissions = pd.read_csv('samplesubmissionv2.csv')
vd = pd.read_csv('variable_descriptions_v2.csv')

In [66]:
print(train.shape)
print(test.shape)

(3174, 40)
(1102, 40)


In [59]:
#train1 = pd.read_csv('Train_maskedv2.csv')
Labels = np.array(train['target_pct_vunerable'])
print(Labels)

[1.73980135 1.99338128 0.03834043 ... 0.95882823 1.02493356 5.28312496]


In [65]:
train = train.drop(['total_households'], axis = 1)
test = test.drop(['total_households'], axis = 1)

In [61]:
#excluded_cols = ['dw_03', 'dw_05', 'pg_02', 'lan_14', 'psa_03', 'lan_02', 'dw_02', 'dw_10', 
               # 'psa_02', 'lan_07', 'lan_09', 'lan_11', 'dw_11', 'lan_10']
    
excluded_cols = ['dw_12', 'dw_13', 'lan_13', 'dw_10', 'psa_02', 'lan_07', 'lan_09']

for i in excluded_cols:
    test = test.drop(i, axis=1)
    
for j in excluded_cols:
    train = train.drop(j, axis=1)

In [62]:
train = train.drop(['target_pct_vunerable'], axis=1)
print(train.shape)
print(test.shape)

(3174, 41)
(1102, 41)


In [63]:
train['avg_individuals'] = train['total_individuals'] / train['total_households']
test['avg_individuals'] = test['total_individuals'] / test['total_households']

In [67]:
Features = np.array(train)
test_enc = np.array(test)

In [47]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(Features[:, :2])
Features[:, :2] = scaler.transform(Features[:, :2])
print(Features[:, :2])

[[-1.63637577 -0.49131626]
 [-2.06257659 -0.50572765]
 [ 0.08052293 -0.50452626]
 ...
 [ 0.21982731 -0.42272933]
 [-0.11136949 -0.49787438]
 [ 0.39675424 -0.44792276]]


In [27]:
scaler1 = preprocessing.StandardScaler().fit(test_enc[:, :2])
test_enc[:, :2] = scaler1.transform(test_enc[:, :2])
print(test_enc[:, :2])

[[-0.16503446 -0.35938299]
 [-0.42163757 -0.42024847]
 [-0.46203328 -0.6051864 ]
 ...
 [ 0.32568337  0.19777147]
 [-0.32950555 -0.10753076]
 [-0.38886785 -0.5674457 ]]


In [68]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(Features.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = Features[indx[0],:]
y_train = np.ravel(Labels[indx[0]])
x_test = Features[indx[1],:]
y_test = np.ravel(Labels[indx[1]])

In [69]:
indx1 = range(x_train.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.2)
x_train1 = x_train[indx1[0],:]
y_train1 = np.ravel(y_train[indx1[0]])
x_train_val = x_train[indx1[1],:]
y_train_val = np.ravel(y_train[indx1[1]])

In [72]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=200, max_depth=10, max_features='auto',
                            min_samples_leaf=1, min_samples_split=2, random_state=0)
rf.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [73]:
rf_pred = rf.predict(x_test)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
mean_squared_error(rf_pred, y_test) ** 0.5

6.779394770375814

In [74]:
preds = xg_reg.predict(test_enc)
submissions.target_pct_vunerable = preds
submissions.to_csv('SouthAfricaCovid4.csv', index=False)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39']
expected f40, f44, f41, f43, f45, f42 in input data

In [75]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 2000, seed = 123, max_depth = 10,
                           learning_rate=0.01, booster = 'dart', base_score = 0.7, subsample = 0.8,
                           reg_lambda = 0.03)

eval_set = [(x_test, y_test)]
xg_reg.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set, verbose=True, early_stopping_rounds = 200)

[0]	validation_0-rmse:15.88186
Will train until validation_0-rmse hasn't improved in 200 rounds.
[1]	validation_0-rmse:15.76226
[2]	validation_0-rmse:15.64107
[3]	validation_0-rmse:15.51587
[4]	validation_0-rmse:15.39697
[5]	validation_0-rmse:15.28336
[6]	validation_0-rmse:15.16058
[7]	validation_0-rmse:15.04043
[8]	validation_0-rmse:14.92535
[9]	validation_0-rmse:14.80846
[10]	validation_0-rmse:14.70040
[11]	validation_0-rmse:14.58300
[12]	validation_0-rmse:14.47675
[13]	validation_0-rmse:14.36789
[14]	validation_0-rmse:14.25675
[15]	validation_0-rmse:14.15052
[16]	validation_0-rmse:14.05161
[17]	validation_0-rmse:13.93848
[18]	validation_0-rmse:13.84119
[19]	validation_0-rmse:13.73847
[20]	validation_0-rmse:13.63832
[21]	validation_0-rmse:13.54280
[22]	validation_0-rmse:13.43924
[23]	validation_0-rmse:13.34153
[24]	validation_0-rmse:13.24998
[25]	validation_0-rmse:13.15116
[26]	validation_0-rmse:13.06063
[27]	validation_0-rmse:12.97297
[28]	validation_0-rmse:12.88228
[29]	validation_

[256]	validation_0-rmse:6.84716
[257]	validation_0-rmse:6.84469
[258]	validation_0-rmse:6.84349
[259]	validation_0-rmse:6.84171
[260]	validation_0-rmse:6.83973
[261]	validation_0-rmse:6.83732
[262]	validation_0-rmse:6.83457
[263]	validation_0-rmse:6.83172
[264]	validation_0-rmse:6.83030
[265]	validation_0-rmse:6.82871
[266]	validation_0-rmse:6.82661
[267]	validation_0-rmse:6.82378
[268]	validation_0-rmse:6.82234
[269]	validation_0-rmse:6.81917
[270]	validation_0-rmse:6.81662
[271]	validation_0-rmse:6.81459
[272]	validation_0-rmse:6.81244
[273]	validation_0-rmse:6.81025
[274]	validation_0-rmse:6.80852
[275]	validation_0-rmse:6.80574
[276]	validation_0-rmse:6.80315
[277]	validation_0-rmse:6.80161
[278]	validation_0-rmse:6.79948
[279]	validation_0-rmse:6.79749
[280]	validation_0-rmse:6.79627
[281]	validation_0-rmse:6.79467
[282]	validation_0-rmse:6.79219
[283]	validation_0-rmse:6.79051
[284]	validation_0-rmse:6.78849
[285]	validation_0-rmse:6.78665
[286]	validation_0-rmse:6.78364
[287]	va

[513]	validation_0-rmse:6.65789
[514]	validation_0-rmse:6.65799
[515]	validation_0-rmse:6.65771
[516]	validation_0-rmse:6.65761
[517]	validation_0-rmse:6.65771
[518]	validation_0-rmse:6.65759
[519]	validation_0-rmse:6.65746
[520]	validation_0-rmse:6.65743
[521]	validation_0-rmse:6.65735
[522]	validation_0-rmse:6.65703
[523]	validation_0-rmse:6.65693
[524]	validation_0-rmse:6.65697
[525]	validation_0-rmse:6.65710
[526]	validation_0-rmse:6.65716
[527]	validation_0-rmse:6.65685
[528]	validation_0-rmse:6.65657
[529]	validation_0-rmse:6.65665
[530]	validation_0-rmse:6.65662
[531]	validation_0-rmse:6.65656
[532]	validation_0-rmse:6.65643
[533]	validation_0-rmse:6.65648
[534]	validation_0-rmse:6.65623
[535]	validation_0-rmse:6.65648
[536]	validation_0-rmse:6.65631
[537]	validation_0-rmse:6.65621
[538]	validation_0-rmse:6.65605
[539]	validation_0-rmse:6.65615
[540]	validation_0-rmse:6.65599
[541]	validation_0-rmse:6.65595
[542]	validation_0-rmse:6.65594
[543]	validation_0-rmse:6.65550
[544]	va

[770]	validation_0-rmse:6.64722
[771]	validation_0-rmse:6.64723
[772]	validation_0-rmse:6.64719
[773]	validation_0-rmse:6.64727
[774]	validation_0-rmse:6.64733
[775]	validation_0-rmse:6.64729
[776]	validation_0-rmse:6.64731
[777]	validation_0-rmse:6.64730
[778]	validation_0-rmse:6.64726
[779]	validation_0-rmse:6.64722
[780]	validation_0-rmse:6.64721
[781]	validation_0-rmse:6.64724
[782]	validation_0-rmse:6.64722
[783]	validation_0-rmse:6.64714
[784]	validation_0-rmse:6.64716
[785]	validation_0-rmse:6.64712
[786]	validation_0-rmse:6.64709
[787]	validation_0-rmse:6.64720
[788]	validation_0-rmse:6.64714
[789]	validation_0-rmse:6.64717
[790]	validation_0-rmse:6.64710
[791]	validation_0-rmse:6.64712
[792]	validation_0-rmse:6.64715
[793]	validation_0-rmse:6.64708
[794]	validation_0-rmse:6.64708
[795]	validation_0-rmse:6.64701
[796]	validation_0-rmse:6.64698
[797]	validation_0-rmse:6.64696
[798]	validation_0-rmse:6.64692
[799]	validation_0-rmse:6.64688
[800]	validation_0-rmse:6.64687
[801]	va

[1026]	validation_0-rmse:6.64455
[1027]	validation_0-rmse:6.64450
[1028]	validation_0-rmse:6.64446
[1029]	validation_0-rmse:6.64447
[1030]	validation_0-rmse:6.64450
[1031]	validation_0-rmse:6.64447
[1032]	validation_0-rmse:6.64444
[1033]	validation_0-rmse:6.64441
[1034]	validation_0-rmse:6.64437
[1035]	validation_0-rmse:6.64435
[1036]	validation_0-rmse:6.64438
[1037]	validation_0-rmse:6.64440
[1038]	validation_0-rmse:6.64436
[1039]	validation_0-rmse:6.64437
[1040]	validation_0-rmse:6.64433
[1041]	validation_0-rmse:6.64432
[1042]	validation_0-rmse:6.64431
[1043]	validation_0-rmse:6.64428
[1044]	validation_0-rmse:6.64427
[1045]	validation_0-rmse:6.64428
[1046]	validation_0-rmse:6.64429
[1047]	validation_0-rmse:6.64427
[1048]	validation_0-rmse:6.64425
[1049]	validation_0-rmse:6.64422
[1050]	validation_0-rmse:6.64418
[1051]	validation_0-rmse:6.64420
[1052]	validation_0-rmse:6.64423
[1053]	validation_0-rmse:6.64420
[1054]	validation_0-rmse:6.64426
[1055]	validation_0-rmse:6.64425
[1056]	val

[1275]	validation_0-rmse:6.64359
[1276]	validation_0-rmse:6.64358
[1277]	validation_0-rmse:6.64358
[1278]	validation_0-rmse:6.64357
[1279]	validation_0-rmse:6.64358
[1280]	validation_0-rmse:6.64357
[1281]	validation_0-rmse:6.64358
[1282]	validation_0-rmse:6.64358
[1283]	validation_0-rmse:6.64357
[1284]	validation_0-rmse:6.64357
[1285]	validation_0-rmse:6.64358
[1286]	validation_0-rmse:6.64359
[1287]	validation_0-rmse:6.64360
[1288]	validation_0-rmse:6.64361
[1289]	validation_0-rmse:6.64362
[1290]	validation_0-rmse:6.64360
[1291]	validation_0-rmse:6.64359
[1292]	validation_0-rmse:6.64360
[1293]	validation_0-rmse:6.64361
[1294]	validation_0-rmse:6.64360
[1295]	validation_0-rmse:6.64359
[1296]	validation_0-rmse:6.64358
[1297]	validation_0-rmse:6.64358
[1298]	validation_0-rmse:6.64357
[1299]	validation_0-rmse:6.64355
[1300]	validation_0-rmse:6.64354
[1301]	validation_0-rmse:6.64356
[1302]	validation_0-rmse:6.64356
[1303]	validation_0-rmse:6.64356
[1304]	validation_0-rmse:6.64356
[1305]	val

[1524]	validation_0-rmse:6.64331
[1525]	validation_0-rmse:6.64331
[1526]	validation_0-rmse:6.64331
[1527]	validation_0-rmse:6.64331
[1528]	validation_0-rmse:6.64331
[1529]	validation_0-rmse:6.64332
[1530]	validation_0-rmse:6.64333
[1531]	validation_0-rmse:6.64333
[1532]	validation_0-rmse:6.64333
[1533]	validation_0-rmse:6.64334
[1534]	validation_0-rmse:6.64334
[1535]	validation_0-rmse:6.64334
[1536]	validation_0-rmse:6.64334
[1537]	validation_0-rmse:6.64334
[1538]	validation_0-rmse:6.64334
[1539]	validation_0-rmse:6.64334
[1540]	validation_0-rmse:6.64334
[1541]	validation_0-rmse:6.64334
[1542]	validation_0-rmse:6.64333
[1543]	validation_0-rmse:6.64333
[1544]	validation_0-rmse:6.64333
[1545]	validation_0-rmse:6.64333
[1546]	validation_0-rmse:6.64333
[1547]	validation_0-rmse:6.64333
[1548]	validation_0-rmse:6.64332
[1549]	validation_0-rmse:6.64332
[1550]	validation_0-rmse:6.64332
[1551]	validation_0-rmse:6.64332
[1552]	validation_0-rmse:6.64333
[1553]	validation_0-rmse:6.64333
[1554]	val

XGBRegressor(base_score=0.7, booster='dart', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.01, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=2000, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=123, reg_alpha=0,
       reg_lambda=0.03, scale_pos_weight=1, seed=123, subsample=0.8,
       tree_method=None, validate_parameters=False, verbosity=None)